# 1.5.9 Feature Matching
## Brute-Force Matching with ORB Descriptors

In [1]:
import numpy as np
import cv2

In [5]:
img1 = cv2.imread('face.png',0)
img2 = cv2.imread('hu.png',0)
img3 = None
orb = cv2.ORB_create()

kp1,des1 = orb.detectAndCompute(img1,None)
kp2,des2 = orb.detectAndCompute(img2,None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

matches = bf.match(des1,des2)

matches = sorted(matches, key = lambda x:x.distance)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches[:10],img3,flags=2)

cv2.imshow("Feature Matching",img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Brute-Force Matching with SIFT Descriptors and Ratio Test

In [7]:
qimg = cv2.imread('face.png',0)
timg = cv2.imread('hu.png',0)
res2 = None

sift = cv2.xfeatures2d.SIFT_create()

kp1, des1 = sift.detectAndCompute(qimg,None)
kp2, des2 = sift.detectAndCompute(timg,None)

bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2,k=2)

good = []

for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
        
res2 = cv2.drawMatchesKnn(qimg,kp1,timg,kp2,good,res2,flags=2)

cv2.imshow("BF with SIFT",res2)
cv2.waitKey(0)
cv2.destroyAllWindows()

## FLANN based Matcher

In [9]:
qimg = cv2.imread('face.png',0)
timg = cv2.imread('hu.png',0)
rimg = cv2.imread('ro_hu.png',0)
res1, res2 = None, None

sift = cv2.xfeatures2d.SIFT_create()

kp1, des1 = sift.detectAndCompute(qimg,None)
kp2, des2 = sift.detectAndCompute(timg,None)
kp3, des3 = sift.detectAndCompute(rimg,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE,trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches1 = flann.knnMatch(des1,des2,k=2)
matches2 = flann.knnMatch(des1,des3,k=2)

matchesMask = [[0,0] for i in range(len(matches))]

for i,(m,m) in enumerate(matches1):
    if m.distance < 0.7*n.distance:
        matchesMask[i] = [1,0]
        
draw_params = dict(matchColor = (0,255,0),
                  singlePointColor = (255,0,0),
                  matchesMask = matchesMask,
                  flags = 0)

res1 = cv2.drawMatchesKnn(qimg,kp1,timg,kp2,matches1,res1,**draw_params)

cv2.imshow('FLANN',res1)
cv2.waitKey(0)
cv2.destroyAllWindows()